# **Alunos:**

**Matheus Vieira Faria** - 20112798

**Daniel Pessoa Máximo** - 20211851

# **Questão 1**

Escolha uma das metodologias que você implementou na segunda
lista para gerar correspondências entre um par de imagens. Aplique-a em
5 pares de imagens (com sobreposição) para calcular suas homografias, e
aplique-as para gerar panoramas entre os pares de imagens (um panorama
por par).
Obs.: nessa questão, não é permitido usar a API de alto nível Stitcher.
Dica: use a função warpPerspective da OpenCV.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import os

# Criar pasta para armazenar imagens
os.makedirs("imagens_q1", exist_ok=True)

uploaded = files.upload()

# Organizar os pares de imagens
image_pairs = []
temp_list = []

for i, filename in enumerate(uploaded.keys()):
    new_path = f"imagens_q1/{filename}"
    os.rename(filename, new_path)
    temp_list.append(new_path)

    if (i+1) % 2 == 0:
        image_pairs.append((temp_list[0], temp_list[1]))
        temp_list = []

def compute_homography_and_panorama(img1, img2):
    # Converter para cinza
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Detectar keypoints e descritores
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    # Casamento de pontos
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Verifica se há matches suficientes
    if len(good_matches) > 4:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Estima a homografia
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        # Cria o panorama
        height1, width1 = img1.shape[:2]
        height2, width2 = img2.shape[:2]

        # Transformar img1 para o espaço da img2
        panorama_corners = cv2.perspectiveTransform(
            np.float32([[0,0],[0,height1],[width1,height1],[width1,0]]).reshape(-1,1,2), H)

        # Calcular o tamanho do panorama resultante
        points = np.concatenate((panorama_corners, np.float32([[0,0],[0,height2],[width2,height2],[width2,0]]).reshape(-1,1,2)), axis=0)
        [xmin, ymin] = np.int32(points.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(points.max(axis=0).ravel() + 0.5)
        translation_dist = [-xmin, -ymin]

        H_translation = np.array([[1, 0, translation_dist[0]],
                                  [0, 1, translation_dist[1]],
                                  [0, 0, 1]])

        result = cv2.warpPerspective(img1, H_translation.dot(H), (xmax - xmin, ymax - ymin))
        result[translation_dist[1]:height2+translation_dist[1], translation_dist[0]:width2+translation_dist[0]] = img2

        return result
    else:
        print("Poucos matches encontrados.")
        return None

# Processar e exibir panoramas
for idx, (img1_path, img2_path) in enumerate(image_pairs, 1):
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    print(f"\nGerando panorama para par {idx}")
    panorama = compute_homography_and_panorama(img1, img2)

    if panorama is not None:
        plt.figure(figsize=(12,6))
        plt.imshow(cv2.cvtColor(panorama, cv2.COLOR_BGR2RGB))
        plt.title(f"Panorama {idx}")
        plt.axis('off')
        plt.show()

# **Questão 2**

Repita a questão anterior com 5 trios de imagens (com sobreposição
2 a 2), alinhando as imagens no plano da primeira imagem. Repita o mesmo
alinhando no plano da segunda imagem, e da terceira imagem. Note que aqui
será necessário compor as transformações de homografia em alguns casos, ou
calcular inversas.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import os

# Criar pasta para armazenar imagens
os.makedirs("imagens_q2", exist_ok=True)

uploaded = files.upload()

# Organizar os trios de imagens
image_trios = []
temp_list = []

for i, filename in enumerate(uploaded.keys()):
    new_path = f"imagens_q2/{filename}"
    os.rename(filename, new_path)
    temp_list.append(new_path)

    if (i+1) % 3 == 0:
        image_trios.append((temp_list[0], temp_list[1], temp_list[2]))
        temp_list = []

def compute_homography_and_panorama(img1, img2):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 4:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        height1, width1 = img1.shape[:2]
        height2, width2 = img2.shape[:2]

        panorama_corners = cv2.perspectiveTransform(
            np.float32([[0,0],[0,height1],[width1,height1],[width1,0]]).reshape(-1,1,2), H)

        points = np.concatenate((panorama_corners, np.float32([[0,0],[0,height2],[width2,height2],[width2,0]]).reshape(-1,1,2)), axis=0)
        [xmin, ymin] = np.int32(points.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(points.max(axis=0).ravel() + 0.5)
        translation_dist = [-xmin, -ymin]

        H_translation = np.array([[1, 0, translation_dist[0]],
                                  [0, 1, translation_dist[1]],
                                  [0, 0, 1]])

        result = cv2.warpPerspective(img1, H_translation.dot(H), (xmax - xmin, ymax - ymin))
        result[translation_dist[1]:height2+translation_dist[1], translation_dist[0]:width2+translation_dist[0]] = img2

        return result
    else:
        print("Poucos matches encontrados.")
        return None


def match_3_on_first(img1, img2, img3):
    inter = compute_homography_and_panorama(img2, img1)
    if inter is not None:
        result = compute_homography_and_panorama(img3, inter)
        if result is not None:
            plt.figure(figsize=(12,6))
            plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
            plt.title("Panorama no plano da 1ª imagem")
            plt.axis("off")
            plt.show()

def match_3_on_second(img1, img2, img3):
    inter = compute_homography_and_panorama(img3, img2)
    if inter is not None:
        result = compute_homography_and_panorama(img1, inter)
        if result is not None:
            plt.figure(figsize=(12,6))
            plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
            plt.title("Panorama no plano da 2ª imagem")
            plt.axis("off")
            plt.show()

def match_3_on_third(img1, img2, img3):
    inter = compute_homography_and_panorama(img2, img3)
    if inter is not None:
        result = compute_homography_and_panorama(img1, inter)
        if result is not None:
            plt.figure(figsize=(12,6))
            plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
            plt.title("Panorama no plano da 3ª imagem")
            plt.axis("off")
            plt.show()

# Processar todos os trios em 3 perspectivas
for idx, (img1_path, img2_path, img3_path) in enumerate(image_trios, 1):
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    img3 = cv2.imread(img3_path)

    print(f"\n==== Panorama para Trio {idx} ====")

    print("→ Alinhando no plano da 1ª imagem")
    match_3_on_first(img1, img2, img3)

    print("→ Alinhando no plano da 2ª imagem")
    match_3_on_second(img1, img2, img3)

    print("→ Alinhando no plano da 3ª imagem")
    match_3_on_third(img1, img2, img3)

# **Questão 3**

Considere a imagem soccer.jpg em anexo no Google Classroom.
Considere que o campo da imagem tenha as dimensões dadas pela figura
abaixo. Gere manualmente correspondências entre a imagem e um mapa 2d com
dimensões dadas pela figura. Calcule a homografia resultante e aplique na
imagem original. Exiba o resultado.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Carregar as imagens
img = cv2.imread('soccer.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
tamanhos_img = cv2.imread('tamanhos.png')
tamanhos_img = cv2.cvtColor(tamanhos_img, cv2.COLOR_BGR2RGB)

# Exibir a imagem original
plt.figure(figsize=(10, 6))
plt.title("Imagem Original do Campo")
plt.imshow(img)
plt.axis('off')
plt.show()

# Definir os pontos de correspondência
img_points = np.array([
    [911, 274],
    [192, 362],
    [371, 601],
    [1188, 479]
], dtype=np.float32)

world_points = np.array([
    [65, 25],
    [65, 428],
    [375, 428],
    [375, 26]
], dtype=np.float32)

# Calcular homografia
H, _ = cv2.findHomography(img_points, world_points)

# Aplicar a transformação
height, width = 485, 775
warped = cv2.warpPerspective(img, H, (width, height))

# Criar figura com subplots
fig, (ax2, ax1) = plt.subplots(1, 2, figsize=(20, 10))

ax2.imshow(tamanhos_img)
ax2.set_title("Referência de Tamanhos")
ax2.grid(True, linestyle='--', alpha=0.5)

ax1.imshow(warped)
ax1.set_title("Vista Superior Transformada")
ax1.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# **Questão 4**

Leia o seguinte tutorial de calibração de câmera:
https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

Você vai precisar de um tabuleiro de xadrez (pode ser impresso em uma folha A4, colado em um papelão ou plastificado para manter a geometria estável). Meça as dimensões reais do tabuleiro para calibrar a câmera, assumindo que ele está no plano z = 0 e que o canto inferior esquerdo do tabuleiro é a origem (0, 0, 0).

Após calibrar a câmera, você deve incluir um objeto virtual na imagem. O objeto será um círculo definido pela seguinte equação paramétrica:

𝑝 (
𝜃
)
=
(
𝑟
⋅
cos
⁡
(
𝜃
)
+
1.5
𝑊 ,
𝑟
⋅
\sen
(
𝜃
)
+
1.5
𝐻
,
0
)
p(θ)=(r⋅cos(θ)+1.5W, r⋅\sen(θ)+1.5H, 0)

Esse círculo está no plano z = 0, com centro no ponto (1.5W, 1.5H, 0) e raio
𝑟
=
0.5
𝑊
r=0.5W, onde
𝑊
W e
𝐻
H são a largura e a altura do tabuleiro, respectivamente.

Certifique-se de que o círculo apareça na imagem de acordo com a posição do tabuleiro. Para desenhá-lo, varie o valor de
𝜃
θ entre 0 e
2
𝜋
2π para gerar os pontos do círculo, e projete esses pontos na imagem.

Repita esse processo 3 vezes, alterando o ângulo entre o vetor normal do tabuleiro e o eixo principal da câmera.

In [ ]:
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow

# ------------------------
# 1. Configurações Iniciais
# ------------------------
chessboard_size = (7, 7)  # Número de cantos INTERNOS
square_size = 2  # Tamanho do quadrado em cm

# Preparar pontos 3D do tabuleiro (objeto)
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size

# Listas para armazenar pontos
objpoints = []  # Pontos 3D no mundo real
imgpoints = []  # Pontos 2D na imagem

# ------------------------
# 2. Carregar Imagens da Pasta
# ------------------------
# Diretório das imagens (ajuste o caminho se necessário)
image_dir = 'Fotos_tabuleiro'
image_names = sorted([f for f in os.listdir(image_dir) if f.startswith('tab')])  # Filtra tab1.jpg, tab2.jpg, etc.

# ------------------------
# 3. Detecção de Cantos e Calibração
# ------------------------
for fname in image_names:
    img_path = os.path.join(image_dir, fname)
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Encontrar cantos do tabuleiro
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)
        corners_refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                         (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        imgpoints.append(corners_refined)

        # Visualizar cantos (apenas para verificação)
        cv2.drawChessboardCorners(img, chessboard_size, corners_refined, ret)
        cv2_imshow(img)

# Calibrar câmera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Matriz da Câmera:\n", mtx)
print("\nCoeficientes de Distorção:\n", dist)

# ------------------------
# 4. Projeção do Círculo Virtual
# ------------------------
def draw_circle(img, mtx, dist, rvec, tvec, W, H):
    radius = 0.3 * W  # Reduzido o tamanho
    center = np.array([1.0 * W, 1.0 * H, 0], dtype=np.float32)  # Posição mais central

    theta = np.linspace(0, 2*np.pi, 50)
    circle_points = np.array([
        [radius * np.cos(t) + center[0], radius * np.sin(t) + center[1], 0]
        for t in theta
    ], dtype=np.float32)

    img_points, _ = cv2.projectPoints(circle_points, rvec, tvec, mtx, dist)
    img_points = img_points.reshape(-1, 2).astype(int)

    for i in range(len(img_points) - 1):
        cv2.line(img, tuple(img_points[i]), tuple(img_points[i+1]), (0, 255, 0), 2)
    cv2.line(img, tuple(img_points[-1]), tuple(img_points[0]), (0, 255, 0), 2)

    return img

# ------------------------
# 5. Aplicação nas Imagens
# ------------------------

def imshow_resized(img, scale_percent=30):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    cv2_imshow(resized)

W = chessboard_size[0] * square_size  # Largura total (cm)
H = chessboard_size[1] * square_size  # Altura total (cm)

for i in range(len(image_names)):
    img_path = os.path.join(image_dir, image_names[i])
    img = cv2.imread(img_path)
    if i < len(rvecs):
        img_with_circle = draw_circle(img, mtx, dist, rvecs[i], tvecs[i], W, H)
        print(f"\nCírculo projetado em {image_names[i]}:")
        imshow_resized(img_with_circle)


Testando outros parâmetros pra visualizar

In [ ]:
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow

# ------------------------
# 1. Configurações Iniciais
# ------------------------
chessboard_size = (7, 7)  # Número de cantos INTERNOS
square_size = 2

# Preparar pontos 3D do tabuleiro (objeto)
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size

# Listas para armazenar pontos
objpoints = []
imgpoints = []

# ------------------------
# 2. Carregar Imagens da Pasta
# ------------------------
image_dir = 'Fotos_tabuleiro'
image_names = sorted([f for f in os.listdir(image_dir) if f.startswith('tab')])  # Filtra tab1.jpg, tab2.jpg, etc.

# ------------------------
# 3. Detecção de Cantos e Calibração
# ------------------------
for fname in image_names:
    img_path = os.path.join(image_dir, fname)
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Encontrar cantos do tabuleiro
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)
        corners_refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                         (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        imgpoints.append(corners_refined)

        # Visualizar cantos
        cv2.drawChessboardCorners(img, chessboard_size, corners_refined, ret)
        cv2_imshow(img)

# Calibrar câmera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Matriz da Câmera:\n", mtx)
print("\nCoeficientes de Distorção:\n", dist)

# ------------------------
# 4. Projeção do Círculo Virtual
# ------------------------
def draw_circle(img, mtx, dist, rvec, tvec, W, H):
    radius_percent = 0.25  # 25% da largura do tabuleiro (antes era 0.3)
    center_x_percent = 0.5  # 50% da largura (totalmente centralizado)
    center_y_percent = 0.5  # 50% da altura

    radius = radius_percent * W
    center = np.array([
        center_x_percent * W,
        center_y_percent * H,
        0
    ], dtype=np.float32)

    # Geração dos pontos do círculo
    theta = np.linspace(0, 2*np.pi, 50)
    circle_points = np.array([
        [radius * np.cos(t) + center[0], radius * np.sin(t) + center[1], 0]
        for t in theta
    ], dtype=np.float32)

    # Projeção e desenho
    img_points, _ = cv2.projectPoints(circle_points, rvec, tvec, mtx, dist)
    img_points = img_points.reshape(-1, 2).astype(int)

    # Desenha o círculo (agora com cor vermelha para melhor visualização)
    for i in range(len(img_points) - 1):
        cv2.line(img, tuple(img_points[i]), tuple(img_points[i+1]), (0, 0, 255), 3)
    cv2.line(img, tuple(img_points[-1]), tuple(img_points[0]), (0, 0, 255), 3)

    return img

# ------------------------
# 5. Aplicação nas Imagens
# ------------------------

def imshow_resized(img, scale_percent=30):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    cv2_imshow(resized)

W = chessboard_size[0] * square_size  # Largura total (cm)
H = chessboard_size[1] * square_size  # Altura total (cm)

for i in range(len(image_names)):
    img_path = os.path.join(image_dir, image_names[i])
    img = cv2.imread(img_path)
    if i < len(rvecs):
        img_with_circle = draw_circle(img, mtx, dist, rvecs[i], tvecs[i], W, H)
        print(f"\nCírculo projetado em {image_names[i]}:")
        imshow_resized(img_with_circle)
